# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4491ff29d0>
├── 'a' --> tensor([[ 1.1997, -0.6782,  0.5326],
│                   [ 0.3010,  1.0589,  0.3923]])
└── 'x' --> <FastTreeValue 0x7f4491ff2b80>
    └── 'c' --> tensor([[-0.3600, -1.2195, -1.1781,  0.0177],
                        [-0.4009, -0.0703,  0.8264, -1.8957],
                        [-0.4698,  1.2763, -0.9830,  1.2150]])

In [4]:
t.a

tensor([[ 1.1997, -0.6782,  0.5326],
        [ 0.3010,  1.0589,  0.3923]])

In [5]:
%timeit t.a

64.6 ns ± 0.0759 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4491ff29d0>
├── 'a' --> tensor([[-0.3606,  1.0201, -1.5347],
│                   [ 1.6226,  0.8683, -0.9375]])
└── 'x' --> <FastTreeValue 0x7f4491ff2b80>
    └── 'c' --> tensor([[-0.3600, -1.2195, -1.1781,  0.0177],
                        [-0.4009, -0.0703,  0.8264, -1.8957],
                        [-0.4698,  1.2763, -0.9830,  1.2150]])

In [7]:
%timeit t.a = new_value

66.1 ns ± 0.0694 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.1997, -0.6782,  0.5326],
               [ 0.3010,  1.0589,  0.3923]]),
    x: Batch(
           c: tensor([[-0.3600, -1.2195, -1.1781,  0.0177],
                      [-0.4009, -0.0703,  0.8264, -1.8957],
                      [-0.4698,  1.2763, -0.9830,  1.2150]]),
       ),
)

In [10]:
b.a

tensor([[ 1.1997, -0.6782,  0.5326],
        [ 0.3010,  1.0589,  0.3923]])

In [11]:
%timeit b.a

59.1 ns ± 0.0733 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2780, -0.2624,  0.5426],
               [ 0.7481, -1.0189,  0.7137]]),
    x: Batch(
           c: tensor([[-0.3600, -1.2195, -1.1781,  0.0177],
                      [-0.4009, -0.0703,  0.8264, -1.8957],
                      [-0.4698,  1.2763, -0.9830,  1.2150]]),
       ),
)

In [13]:
%timeit b.a = new_value

497 ns ± 0.165 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

822 ns ± 0.694 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 401 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 519 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f43e3242a60>
├── 'a' --> tensor([[[ 1.1997, -0.6782,  0.5326],
│                    [ 0.3010,  1.0589,  0.3923]],
│           
│                   [[ 1.1997, -0.6782,  0.5326],
│                    [ 0.3010,  1.0589,  0.3923]],
│           
│                   [[ 1.1997, -0.6782,  0.5326],
│                    [ 0.3010,  1.0589,  0.3923]],
│           
│                   [[ 1.1997, -0.6782,  0.5326],
│                    [ 0.3010,  1.0589,  0.3923]],
│           
│                   [[ 1.1997, -0.6782,  0.5326],
│                    [ 0.3010,  1.0589,  0.3923]],
│           
│                   [[ 1.1997, -0.6782,  0.5326],
│                    [ 0.3010,  1.0589,  0.3923]],
│           
│                   [[ 1.1997, -0.6782,  0.5326],
│                    [ 0.3010,  1.0589,  0.3923]],
│           
│                   [[ 1.1997, -0.6782,  0.5326],
│                    [ 0.3010,  1.0589,  0.3923]]])
└── 'x' --> <FastTreeValue 0x7f448cf82b50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.3 µs ± 56.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f446442dc10>
├── 'a' --> tensor([[ 1.1997, -0.6782,  0.5326],
│                   [ 0.3010,  1.0589,  0.3923],
│                   [ 1.1997, -0.6782,  0.5326],
│                   [ 0.3010,  1.0589,  0.3923],
│                   [ 1.1997, -0.6782,  0.5326],
│                   [ 0.3010,  1.0589,  0.3923],
│                   [ 1.1997, -0.6782,  0.5326],
│                   [ 0.3010,  1.0589,  0.3923],
│                   [ 1.1997, -0.6782,  0.5326],
│                   [ 0.3010,  1.0589,  0.3923],
│                   [ 1.1997, -0.6782,  0.5326],
│                   [ 0.3010,  1.0589,  0.3923],
│                   [ 1.1997, -0.6782,  0.5326],
│                   [ 0.3010,  1.0589,  0.3923],
│                   [ 1.1997, -0.6782,  0.5326],
│                   [ 0.3010,  1.0589,  0.3923]])
└── 'x' --> <FastTreeValue 0x7f43e2bffdf0>
    └── 'c' --> tensor([[-0.3600, -1.2195, -1.1781,  0.0177],
                        [-0.4009, -0.0703,  0.8264, -1.8957],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 58.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.6 µs ± 99.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.3600, -1.2195, -1.1781,  0.0177],
                       [-0.4009, -0.0703,  0.8264, -1.8957],
                       [-0.4698,  1.2763, -0.9830,  1.2150]],
              
                      [[-0.3600, -1.2195, -1.1781,  0.0177],
                       [-0.4009, -0.0703,  0.8264, -1.8957],
                       [-0.4698,  1.2763, -0.9830,  1.2150]],
              
                      [[-0.3600, -1.2195, -1.1781,  0.0177],
                       [-0.4009, -0.0703,  0.8264, -1.8957],
                       [-0.4698,  1.2763, -0.9830,  1.2150]],
              
                      [[-0.3600, -1.2195, -1.1781,  0.0177],
                       [-0.4009, -0.0703,  0.8264, -1.8957],
                       [-0.4698,  1.2763, -0.9830,  1.2150]],
              
                      [[-0.3600, -1.2195, -1.1781,  0.0177],
                       [-0.4009, -0.0703,  0.8264, -1.8957],
                       [-0.4698,  1.2763, -0.9830,  1.2150]],

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.3600, -1.2195, -1.1781,  0.0177],
                      [-0.4009, -0.0703,  0.8264, -1.8957],
                      [-0.4698,  1.2763, -0.9830,  1.2150],
                      [-0.3600, -1.2195, -1.1781,  0.0177],
                      [-0.4009, -0.0703,  0.8264, -1.8957],
                      [-0.4698,  1.2763, -0.9830,  1.2150],
                      [-0.3600, -1.2195, -1.1781,  0.0177],
                      [-0.4009, -0.0703,  0.8264, -1.8957],
                      [-0.4698,  1.2763, -0.9830,  1.2150],
                      [-0.3600, -1.2195, -1.1781,  0.0177],
                      [-0.4009, -0.0703,  0.8264, -1.8957],
                      [-0.4698,  1.2763, -0.9830,  1.2150],
                      [-0.3600, -1.2195, -1.1781,  0.0177],
                      [-0.4009, -0.0703,  0.8264, -1.8957],
                      [-0.4698,  1.2763, -0.9830,  1.2150],
                      [-0.3600, -1.2195, -1.1781,  0.0177],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 254 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

322 µs ± 573 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
